In [1]:
print('Initializing data engineering!')
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import json, requests

### World data engineering
#### Fetching worldwide data

In [2]:
# df = pd.read_json('https://pomber.github.io/covid19/timeseries.json')
# df = pd.read_json('https://covidapi.info/api/v1/country/BRA')
# df = pd.read_json('https://api.covid19api.com/dayone/country/brazil/status/confirmed')
#https://documenter.getpostman.com/view/10808728/SzS8rjbc?version=latest#cc76052f-6601-4645-80e5-ca7aaa36f8ef
df_countries = pd.read_csv('../data/world_countries_2019.csv')

df = pd.DataFrame()
url = "https://pomber.github.io/covid19/timeseries.json"
req = requests.get(url)
# r = r.json()
j = json.loads(req.text)

#### Fetching countries's pandemic data from Pomber's JSON to a dataframe 

In [3]:
# Loading countries names to dict
countries = []
df = pd.DataFrame()
for country in j:
    countries.append(country)

df['country'] = pd.Series(countries)

# Loading countries data do dict then to dataframe
dic = []
for country in countries:
    i = 0
    while i < len(j[country]):
        if j[country][i]['confirmed'] == 0:
            i += 1
            continue
        row = {'country': country, 'date': j[country][i]['date'], 'cases':j[country][i]['confirmed'],
               'deaths':j[country][i]['deaths'], 'recoveries':j[country][i]['recovered']}
        dic.append(row)
        i += 1 
df = pd.DataFrame.from_dict(dic)
df[df['country']=='France'].tail()

,country,date,cases,deaths,recoveries
2790,France,2020-4-15,134582,17188,31470
2791,France,2020-4-16,147091,17941,33327
2792,France,2020-4-17,149130,18703,35006
2793,France,2020-4-18,149149,19345,36587
2794,France,2020-4-19,154097,19744,37183


#### Feature engineering

In [4]:
for country in countries:
    qtdeDays = len(df[df.country == country])+1
    df.loc[df.country == country, 'day'] = (np.arange(1,qtdeDays,1))
#     df.drop(df[case].index, inplace=True)
    # valores diários
    df.loc[df.country == country, 'case_day'] = df[df.country == country]['cases'].diff()    
    df.loc[df.country == country, 'death_day'] = df[df.country == country]['deaths'].diff()
    df.loc[df.country == country, 'recovery_day'] = df[df.country == country]['recoveries'].diff()

    # % daily variations
    df.loc[df.country == country, '%var_case_day'] = ((df[df.country == country]['case_day'] - df[df.country == country]['case_day'].shift()) / df[df.country == country]['case_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
    df.loc[df.country == country, '%var_death_day'] = ((df[df.country == country]['death_day'] - df[df.country == country]['death_day'].shift()) / df[df.country == country]['death_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
    df.loc[df.country == country, '%var_recovery_day'] = ((df[df.country == country]['recovery_day'] - df[df.country == country]['recovery_day'].shift()) / df[df.country == country]['recovery_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
    
    # Igualo o valor da primeira linha igual ao primeiro número do acumulado, pois se o acumulado começa em 1 o primeiro diff fica igual a 0
    df.loc[(df.country == country) & (df.day == 1), 'case_day']= df.loc[(df.country == country) & (df.day==1), 'cases']
    df.loc[(df.country == country) & (df.day == 1), 'death_day']= df.loc[(df.country == country) & (df.day==1), 'deaths']
    df.loc[(df.country == country) & (df.day == 1), 'recovery_day']= df.loc[(df.country == country) & (df.day==1), 'recoveries']
    
    # Buscando a população do país e calculado os indicador per milhão
    if not df_countries[df_countries['country']==country].empty:
        million = df_countries[df_countries['country']==country]['PopTotal'].values[0] / 1000
        cases_million = (df[df.country == country]['case_day'] / million).round(1)
        deaths_million = (df[df.country == country]['death_day'] / million).round(1)
        recoveries_million = (df[df.country == country]['recovery_day'] / million).round(1)
    else:
        cases_million = 0
        deaths_million = 0
        recoveries_million = 0
        
    df.loc[df.country == country, 'cases_million'] = cases_million
    df.loc[df.country == country, 'deaths_million'] = deaths_million
    df.loc[df.country == country, 'recoveries_million'] = recoveries_million
    
    # moving averages (from last 7 days)
    df.loc[df.country == country, 'avg7_cases'] = df[df.country == country]['case_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    df.loc[df.country == country, 'avg7_deaths'] = df[df.country == country]['death_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    df.loc[df.country == country, 'avg7_recoveries'] = df[df.country == country]['recovery_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    
    df.loc[df.country == country, 'avg7_cases_million'] = df[df.country == country]['cases_million'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    df.loc[df.country == country, 'avg7_deaths_million'] = df[df.country == country]['deaths_million'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    df.loc[df.country == country, 'avg7_recoveries_million'] = df[df.country == country]['recoveries_million'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    

df['perc_death'] = (df['deaths']/df['cases'] * 100).round(2)
df['perc_recovery'] = (df['recoveries']/df['cases'] * 100).round(2)
df['active_cases'] = df['cases'] - df['recoveries'] - df['deaths']

df.fillna(0, inplace=True)

df['day'] = df['day'].astype('int')
df['case_day'] = df['case_day'].astype('int')
df['death_day'] = df['death_day'].astype('int')
df['recovery_day'] = df['recovery_day'].astype('int')

df.tail()

,country,date,cases,deaths,recoveries,day,case_day,death_day,recovery_day,%var_case_day,...,recoveries_million,avg7_cases,avg7_deaths,avg7_recoveries,avg7_cases_million,avg7_deaths_million,avg7_recoveries_million,perc_death,perc_recovery,active_cases
8713,Yemen,2020-4-15,1,0,0,6,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
8714,Yemen,2020-4-16,1,0,0,7,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
8715,Yemen,2020-4-17,1,0,0,8,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
8716,Yemen,2020-4-18,1,0,0,9,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
8717,Yemen,2020-4-19,1,0,0,10,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


#### Saving CSV

In [5]:
df.to_csv('../data/world_corona19_data.csv', index = False)

In [6]:
df[df['country']=='France'].tail()

,country,date,cases,deaths,recoveries,day,case_day,death_day,recovery_day,%var_case_day,...,recoveries_million,avg7_cases,avg7_deaths,avg7_recoveries,avg7_cases_million,avg7_deaths_million,avg7_recoveries_million,perc_death,perc_recovery,active_cases
2790,France,2020-4-15,134582,17188,31470,83,3221,1440,2372,-46.02,...,36.4,7360.0,900.0,1431.0,113.0,13.0,21.0,12.77,23.38,85924
2791,France,2020-4-16,147091,17941,33327,84,12509,753,1857,288.36,...,28.5,8532.0,816.0,1416.0,131.0,12.0,21.0,12.20,22.66,95823
2792,France,2020-4-17,149130,18703,35006,85,2039,762,1679,-83.70,...,25.8,8198.0,784.0,1401.0,125.0,12.0,21.0,12.54,23.47,95421
2793,France,2020-4-18,149149,19345,36587,86,19,642,1581,-99.07,...,24.3,7755.0,784.0,1417.0,119.0,12.0,21.0,12.97,24.53,93217
2794,France,2020-4-19,154097,19744,37183,87,4948,399,596,25942.11,...,9.2,4626.0,761.0,1387.0,71.0,11.0,21.0,12.81,24.13,97170


#### countries not located in UN dataset

In [7]:
for country in countries:
    if df_countries[df_countries['country']==country]['PopTotal'].empty:
        print(country)
# df[df['pais'] == pais].tail()
# df_countries[df_countries['Location']==pais]['PopTotal']

Bosnia and Herzegovina
Brunei
Congo (Brazzaville)
Congo (Kinshasa)
Cote d'Ivoire
Diamond Princess
Eswatini
Holy See
Iran
Korea, South
Moldova
North Macedonia
Russia
Taiwan*
Tanzania
United Arab Emirates
Venezuela
Vietnam
Syria
Laos
Libya
West Bank and Gaza
Kosovo
Burma
MS Zaandam


### Brazil data engineering

In [8]:
df = pd.read_csv('../data/gov_brazil_corona19_data.csv', sep=';')
df.rename(columns={'regiao': 'region', 'estado':'state', 'data':'date','casosNovos': 'case_day', 'casosAcumulados':'cases', 'obitosNovos':'death_day','obitosAcumulados':'deaths'}, inplace= True)

df.tail()

,region,state,date,case_day,cases,death_day,deaths
2182,Centro-Oeste,DF,15/04/2020,31,682,0,17
2183,Centro-Oeste,DF,16/04/2020,34,716,3,20
2184,Centro-Oeste,DF,17/04/2020,30,746,0,20
2185,Centro-Oeste,DF,18/04/2020,16,762,4,24
2186,Centro-Oeste,DF,19/04/2020,65,827,0,24


#### Feature engineering

In [9]:
states = df.state.unique()
df.drop(df[df['cases'] == 0 ].index, axis=0, inplace= True)

for state in states:
    qtdeDays = len(df[df.state == state])+1
    df.loc[df.state == state, 'day'] = (np.arange(1,qtdeDays,1))
#     df.drop(df[case].index, inplace=True)
    # valores diários
    df.loc[df.state == state, 'case_day'] = df[df.state == state]['cases'].diff()    
    df.loc[df.state == state, 'death_day'] = df[df.state == state]['deaths'].diff()
#     df.loc[df.state == state, 'recovery_day'] = df[df.state == state]['recoveries'].diff()

    # % daily variations
    df.loc[df.state == state, '%var_case_day'] = ((df[df.state == state]['case_day'] - df[df.state == state]['case_day'].shift()) / df[df.state == state]['case_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
    df.loc[df.state == state, '%var_death_day'] = ((df[df.state == state]['death_day'] - df[df.state == state]['death_day'].shift()) / df[df.state == state]['death_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
#     df.loc[df.state == state, '%var_recovery_day'] = ((df[df.state == state]['recovery_day'] - df[df.state == state]['recovery_day'].shift()) / df[df.state == state]['recovery_day'].shift()*100).replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)
    
    # Igualo o valor da primeira linha igual ao primeiro número do acumulado, pois se o acumulado começa em 1 o primeiro diff fica igual a 0
    df.loc[(df.state == state) & (df.day == 1), 'case_day']= df.loc[(df.state == state) & (df.day==1), 'cases']
    df.loc[(df.state == state) & (df.day == 1), 'death_day']= df.loc[(df.state == state) & (df.day==1), 'deaths']
    
    # moving averages (from last 7 days)
    df.loc[df.state == state, 'avg7_cases'] = df[df.state == state]['case_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    df.loc[df.state == state, 'avg7_deaths'] = df[df.state == state]['death_day'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).astype('int')
    df.loc[df.state == state, 'perc_death'] = (df[df.state == state]['deaths']/df[df.state == state]['cases']*100).round(2) 
    df.loc[df.state == state, 'avg7_perc_death'] = df[df.state == state]['perc_death'].rolling(window=7).mean().replace([np.inf, -np.inf], 0).replace([np.nan], 0).round(2)

df['perc_death'] = (df['deaths']/df['cases'] * 100).round(2)
# df['perc_recovery'] = (df['recoveries']/df['cases'] * 100).round(2)
# df['active_cases'] = df['cases'] - df['recoveries'] - df['deaths']

df.fillna(0, inplace=True)

df['day'] = df['day'].astype('int')
df['case_day'] = df['case_day'].astype('int')
df['death_day'] = df['death_day'].astype('int')
# df['recovery_day'] = df['recovery_day'].astype('int')

df.tail()

,region,state,date,case_day,cases,death_day,deaths,day,%var_case_day,%var_death_day,avg7_cases,avg7_deaths,perc_death,avg7_perc_death
2182,Centro-Oeste,DF,15/04/2020,31,682,0,17,40,138.46,-100.0,24.0,0.0,2.49,2.45
2183,Centro-Oeste,DF,16/04/2020,34,716,3,20,41,9.68,0.0,27.0,1.0,2.79,2.49
2184,Centro-Oeste,DF,17/04/2020,30,746,0,20,42,-11.76,-100.0,27.0,0.0,2.68,2.52
2185,Centro-Oeste,DF,18/04/2020,16,762,4,24,43,-46.67,0.0,26.0,1.0,3.15,2.62
2186,Centro-Oeste,DF,19/04/2020,65,827,0,24,44,306.25,-100.0,30.0,1.0,2.90,2.71


In [10]:
df.to_csv('../data/brazil_corona19_data.csv', index = False)

In [11]:
# df[df['country']=='Belgium']
print('Data engineering done!')

Data engineering done!
